<a href="https://colab.research.google.com/github/omer358/ml-acamey/blob/master/Copy_of_IFD_VGG_classification_model_MICC2000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf 
from tensorflow.keras import models
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, RandomFlip, RandomRotation, Resizing, Rescaling
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import random
import glob

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# os.chdir(r'/content/drive/MyDrive/Datasets/')
# !unrar x "/content/drive/MyDrive/Datasets/comofod_small.rar"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/Datasets/comofod_small.rar


KeyboardInterrupt: ignored

Data set spliting : tranning-70% - Validation-20% - test-10%

In [ ]:
os.chdir(r'/content/drive/MyDrive/Datasets/CoMoFoD_small_v2')

if os.path.isdir('train/forged') is False:
    os.makedirs('train/forged')
    os.makedirs('train/original')
    
    os.makedirs('val/forged')
    os.makedirs('val/original') 
    
    os.makedirs('test/forged')
    os.makedirs('test/original') 
    
    for img in random.sample(glob.glob('*[F]*'), 3500):
        shutil.move(img, 'train/forged')
    for img in random.sample(glob.glob('*[O]*'), 3500): 
        shutil.move(img, 'train/original')
        
    for img in random.sample(glob.glob('*[F]*'), 1000):
        shutil.move(img, 'val/forged')
    for img in random.sample(glob.glob('*[O]*'), 1000): 
        shutil.move(img, 'val/original')
        
    for img in random.sample(glob.glob('*[F]*'), 500):
        shutil.move(img, 'test/forged')
    for img in random.sample(glob.glob('*[O]*'), 500): 
        shutil.move(img, 'test/original')
        
os.chdir(r'/content')

In [ ]:
!nvidia-smi

Thu Nov  3 08:57:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !unrar x "/content/drive/MyDrive/Datasets/comofod_small.rar"

Streaming output truncated to the last 5000 lines.
Extracting  CoMoFoD_small_v2/104_O_JC8.jpg                                51%  OK 
Extracting  CoMoFoD_small_v2/104_O_JC9.jpg                                51%  OK 
Extracting  CoMoFoD_small_v2/104_O_NA1.png                                51%  OK 
Extracting  CoMoFoD_small_v2/104_O_NA2.png                                51%  OK 
Extracting  CoMoFoD_small_v2/104_O_NA3.png                                51%  OK 
Extracting  CoMoFoD_small_v2/105_B.png                                    51%  OK 
Extracting  CoMoFoD_small_v2/105_F.png                                    52%  OK 
Extracting  CoMoFoD_small_v2/105_F_BC1.png                                52%  OK 
Extracting  CoMoFoD_small_v2/105_F_BC2.png                                52%  OK 
Extracting  CoMoFoD_small_v2/105_F_BC3.png                                52%  OK 
Extracting  CoMoFoD_small_v2/

## Data Preprocessing

In [ ]:
train_path = r'/content/drive/MyDrive/Datasets/CoMoFoD_small_v2/train'
val_path = r'/content/drive/MyDrive/Datasets/CoMoFoD_small_v2/val'
test_path = r'/content/drive/MyDrive/Datasets/CoMoFoD_small_v2/test'

In [ ]:
BATCH_SIZE = 100
IMAGE_SIZE = 244
CHANNELS = 3

The VGG paper states that the only preprocessing that is done is to subtract the mean value of the computed tranning set from the total value.

In [ ]:
train_datagen = ImageDataGenerator(
#     preprocessing_function = tf.keras.applications.vgg16.preprocess_input, 
    rescale= 1./255,
    rotation_range=10,
    horizontal_flip=True
)
       
train_batch = train_datagen.flow_from_directory(
    directory= train_path,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
   class_mode="sparse", batch_size=BATCH_SIZE)

Found 1216 images belonging to 2 classes.


In [ ]:
class_names = list(train_batch.class_indices.keys())
class_names

['forged', 'original']

In [ ]:
for image_batch, label_batch in train_batch:
    print(image_batch[0])
    break

[[[0.16163465 0.1757145  0.1288447 ]
  [0.1694922  0.18517847 0.12801005]
  [0.21282175 0.23072581 0.1563777 ]
  ...
  [0.6464954  0.6279709  0.6011915 ]
  [0.6372568  0.6193481  0.5907211 ]
  [0.6280181  0.6107254  0.58025056]]

 [[0.14993232 0.16339627 0.11960602]
  [0.17565133 0.1913376  0.13478507]
  [0.19619213 0.2134803  0.14344355]
  ...
  [0.5946098  0.5789235  0.5436294 ]
  [0.5890666  0.57338035 0.53808624]
  [0.5835234  0.5678371  0.532543  ]]

 [[0.13823001 0.15107803 0.11036734]
  [0.18181042 0.1974967  0.14156011]
  [0.17956252 0.19623478 0.13050942]
  ...
  [0.58243114 0.56674486 0.53145075]
  [0.5910539  0.5753676  0.5400735 ]
  [0.59967667 0.5839904  0.5486963 ]]

 ...

 [[0.64332795 0.65236086 0.6787837 ]
  [0.6334734  0.644354   0.67324054]
  [0.6236188  0.6363471  0.66769737]
  ...
  [0.6597426  0.66366416 0.683272  ]
  [0.6503798  0.65430135 0.6739092 ]
  [0.6544903  0.65841186 0.6780197 ]]

 [[0.6326697  0.65001106 0.6803399 ]
  [0.6419084  0.6598657  0.68834674]


In [ ]:
# plt.figure(figsize=(10,10))
# for images in train_batch[0] :
#     for i in range(12):
#         #displaying images as grid of 4 rows and 3 columns
#         ax = plt.subplot(3, 4, i+1)
#         plt.imshow(images[i])
#     #showing image lable 
#         img_label = int(labels[i])
#         plt.title("img")
    
#     #disabling the axis 
#         plt.axis("off")

In [ ]:
val_dategen = ImageDataGenerator(
#     preprocessing_function = tf.keras.applications.vgg16.preprocess_input, 
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True
)

val_batch= val_dategen.flow_from_directory(
    directory= val_path, 
    target_size=(IMAGE_SIZE,IMAGE_SIZE), 
    class_mode="sparse", 
    batch_size=BATCH_SIZE)

Found 0 images belonging to 2 classes.


In [ ]:
for image_batch, label_batch in val_batch:
    print(image_batch[0])
    break

IndexError: ignored

In [ ]:

test_datagen = ImageDataGenerator(
#     preprocessing_function = tf.keras.applications.vgg16.preprocess_input, 
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True
)
test_batch = test_datagen.flow_from_directory(
    directory= test_path, target_size=(IMAGE_SIZE,IMAGE_SIZE), 
    class_mode="sparse", batch_size=BATCH_SIZE, shuffle=False)

Found 0 images belonging to 2 classes.


In [ ]:
for image_batch, label_batch in test_batch:
    print(image_batch[0])
    break

IndexError: ignored

## Building The Model Architecture

# VGG-16 Arch

![image.png](attachment:image.png)

In [ ]:
# input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3)
# model = models.Sequential()

# #Block1
# model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", name='B1_conv1', activation="relu"))
# model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", name='B1_conv2', activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='B1_maxpool'))

# #Block2
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", name='B2_conv1', activation="relu"))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", name='B2_conv2', activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='B2_maxpool'))

# #Block3
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", name='B3_conv1', activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", name='B3_conv2', activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", name='B3_conv3', activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B3_maxpool'))

# #Block4
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", name='B4_conv1', activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", name='B4_conv2', activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", name='B4_conv3', activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B4_maxpool'))

# #Block5
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", name='B5_conv1', activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", name='B5_conv2', activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B5_maxpool'))

# #Block6-Classification
# model.add(Flatten())
# model.add(Dense(units=4096,activation="relu", name='fc_1'))
# model.add(Dropout(0.5, name='dropout_1'))

# model.add(Dense(units=4096,activation="relu", name='fc_2'))
# model.add(Dropout(0.5, name='dropout_2'))

# model.add(Dense(units=2, activation="softmax", name='output'))

# model.build(input_shape=input_shape)

# Using Custom CNN Architechture 

In [ ]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

model = models.Sequential()
model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3,3), padding="same", name='B1_conv1', activation="relu"))
model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3,3), padding="same", name='B1_conv2', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B1_maxpool'))

model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3,3), padding="same", name='B2_conv1', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B2_maxpool'))

model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3,3), padding="same", name='B3_conv2', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B3_maxpool'))

model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3,3), padding="same", name='B4_conv1', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B4_maxpool'))

model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3,3), padding="same", name='B5_conv1', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B5_maxpool'))

model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3,3), padding="same", name='B6_conv', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='B6_maxpool'))

model.add(Flatten())
model.add(Dense(units=128,activation="relu", name='fc_1'))
model.add(Dense(units=2,activation="softmax", name='output'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 B1_conv1 (Conv2D)           (None, 244, 244, 128)     3584      
                                                                 
 B1_conv2 (Conv2D)           (None, 244, 244, 128)     147584    
                                                                 
 B1_maxpool (MaxPooling2D)   (None, 122, 122, 128)     0         
                                                                 
 B2_conv1 (Conv2D)           (None, 122, 122, 128)     147584    
                                                                 
 B2_maxpool (MaxPooling2D)   (None, 61, 61, 128)       0         
                                                                 
 B3_conv2 (Conv2D)           (None, 61, 61, 128)       147584    
                                                                 
 B3_maxpool (MaxPooling2D)   (None, 30, 30, 128)       0

## Model Compiling & Tranning

#### Compiling the Model
We use `adam` Optimizer, `SparseCategoricalCrossentropy` for losses, `accuracy` as a metric

In [ ]:
model.compile(
     optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# train_batch = np.asarray(train_batch)
# val_batch = np.asarray(val_batch)

verbose=0 will show you nothing (silent)
verbose=1 will show you an animated progress bar like this: progres_bar [==================]
verbose=2 will just mention the number of epoch like this:

In [ ]:
model.fit(train_batch, validation_data = val_batch, epochs=10 ,validation_steps=6, batch_size=100, verbose=1)

Epoch 1/10
13/13 [==============================] - 39s 2s/step - loss: 0.1091 - accuracy: 1.0000
Epoch 2/10
13/13 [==============================] - 31s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
13/13 [==============================] - 30s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
13/13 [==============================] - 29s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
13/13 [==============================] - 30s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 28s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
13/13 [==============================] - 29s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
13/13 [==============================] - 28s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
13/13 [==============================] - 29s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
13/13 [==============================] - 28s 2s/step - loss: 0.0000e+00 - 

In [ ]:
scores = model.evaluate(test_ds)